#### 산업수학 3주차 과제 (202021186 명서준)

# 추천시스템과 SVD

<br>
<br>

* ***SVD 활용 예시 : 추천시스템*** <br>
추천시스템에서 SVD가 어떻게 활용되는지 발표해보려고합니다.

<br>

목차
1. Collaborative Filtering
2. SVD를 활용한 간단한 추천 시스템 구현
3. 잠재적 요인 기반 추천시스템 구현


***

### 1. Collaborative Filtering
<br>
협업 필터링(Collaborative Filtering)이란<br>
사용자와 아이템 간의 상관관계를 분석하여 새로운 사용자-아이템 관계를 찾아주는<br>
사용자의 과거 경험과 행동방식에 의존하여 추천하는 시스템입니다.<br>

<br>
* Collaborative Filtering(CF)의 접근방법<br>
1. Memory based approach (ex. 코사인 유사도) <br>
2. Model based approach (ex. svd) <br>

<br>
(자세한 내용은 ppt에)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from sklearn.metrics import mean_squared_error

***

### 2. SVD를 활용한 간단한 추천시스템 구현
<br>

In [2]:
# 6명의 구매자, 5개 상품 0~5점 만족도
A = np.array([
    [4,3,5,0,0],
    [3,4,4,0,0],
    [4,5,4,0,0],
    [0,1,0,3,4],
    [1,0,0,4,5],
    [0,0,0,5,4]
])

print(A)

[[4 3 5 0 0]
 [3 4 4 0 0]
 [4 5 4 0 0]
 [0 1 0 3 4]
 [1 0 0 4 5]
 [0 0 0 5 4]]


In [3]:
# A에 대해 SVD 적용
u, s, vt = np.linalg.svd(A)

In [4]:
print(u.shape)
u.round(3)

(6, 6)


array([[-0.567,  0.105, -0.7  ,  0.089, -0.227, -0.344],
       [-0.521,  0.095,  0.192, -0.173, -0.389,  0.708],
       [-0.612,  0.107,  0.466,  0.006,  0.566, -0.278],
       [-0.109, -0.472,  0.44 ,  0.221, -0.596, -0.41 ],
       [-0.123, -0.61 , -0.202,  0.579,  0.322,  0.364],
       [-0.079, -0.611, -0.148, -0.76 ,  0.138, -0.046]])

In [5]:
s

array([12.11827141, 10.24265114,  1.79247471,  1.22245899,  0.72679026])

In [6]:
# s값을 d행렬로 다시 재구성
d = np.zeros((6,5))
for i in range(5) :
    d[i,i] = s[i]
d

array([[12.11827141,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , 10.24265114,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.79247471,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.22245899,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.72679026],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [7]:
vt

array([[-0.52810648, -0.57360392, -0.60769494, -0.10019994, -0.1129111 ],
       [ 0.05131281,  0.0742802 ,  0.13047499, -0.67476139, -0.7207793 ],
       [-0.31233638,  0.80334462, -0.48298683, -0.12513452,  0.09026894],
       [ 0.36069968, -0.14160705, -0.18181276, -0.67055812,  0.60592005],
       [ 0.70057996, -0.00677168, -0.58936489,  0.26334425, -0.30404063]])

In [8]:
# 고윳값 제일 높은 2개로 다시 행렬 A 재생성 -> A의 정보 어느정도 잘 재생

A_new = u[:,0:2]@d[0:2,0:2]@vt[0:2,:]
A_new = A_new.round(2)
A_new

array([[ 3.68,  4.02,  4.32, -0.04, -0.  ],
       [ 3.38,  3.69,  3.96, -0.02,  0.01],
       [ 3.97,  4.33,  4.65,  0.  ,  0.04],
       [ 0.45,  0.4 ,  0.18,  3.39,  3.63],
       [ 0.47,  0.39,  0.09,  4.37,  4.67],
       [ 0.18,  0.08, -0.24,  4.32,  4.62]])

In [9]:
# 행렬의 행은 사용자를 표현하고, 열은 두개의 내재된 성질에 대한 것이다.
# 즉 사용자는 2개의 그룹으로 나누어지며, 
# 1~3번 사용자는 첫번째 성질에 속하고, 4~6번 사용자는 두번째 성질에 속하는 정도가 크다

u[:,0:2]@d[0:2,0:2]

array([[-6.87171239,  1.08046681],
       [-6.30951489,  0.97295921],
       [-7.41122529,  1.09855222],
       [-1.32584816, -4.83312114],
       [-1.49346175, -6.25162922],
       [-0.95264411, -6.25692412]])

In [10]:
# 5개의 상품(각 열)에 대해 첫번째 성질(사용자 그룹)은 1~3열의 상품에 대한 선호도가 높고, 2번째 성질은 4~5열의 상품에 대한 선호도가 높다.
vt[0:2,:]

array([[-0.52810648, -0.57360392, -0.60769494, -0.10019994, -0.1129111 ],
       [ 0.05131281,  0.0742802 ,  0.13047499, -0.67476139, -0.7207793 ]])

In [12]:
# 이렇게 SVD를 이용하면 데이터에 내재되어 있는 의미 파악이 가능하다. 
# 이를 이용하면 상품 추천시스템을 만들 수 있다.

# 이제 어떤 사람의 구매 만족도를 조사하였더니 아래와 같은 데이터가 주어졌다.
# B=[3 ? 4 ? 2]
# 이 사용자는 상품 2, 4를 아직 구매하지 않았다.
# 이 사람에게 상품 2와 4 중에서 어떤 것을 추천할 것인지 계산해 보자.

# 먼저 평가가 있는 세개의 상품에 대해 평균을 구해 '?'위치에 대치하자.
# (3+4+2)/3=3이므로 3을 대입한다.

B = np.array([3,3,4,3,2])

# 이 사용자를 A행렬의 마지막 행에 추가하고 SVD를 다시 수행한다.

A = np.vstack([A,B])
A

array([[4, 3, 5, 0, 0],
       [3, 4, 4, 0, 0],
       [4, 5, 4, 0, 0],
       [0, 1, 0, 3, 4],
       [1, 0, 0, 4, 5],
       [0, 0, 0, 5, 4],
       [3, 3, 4, 3, 2]])

In [13]:
u, s, vt = np.linalg.svd(A)

d = np.zeros((7,5))
for i in range(5) :
    d[i,i] = s[i]

In [14]:
A_new = u[:,0:2]@d[0:2,0:2]@vt[0:2,:]
A_new = A_new.round(2)
A_new

array([[ 3.66,  3.95,  4.43,  0.1 , -0.11],
       [ 3.35,  3.61,  4.06,  0.09, -0.1 ],
       [ 3.92,  4.23,  4.75,  0.13, -0.09],
       [ 0.42,  0.33,  0.23,  3.47,  3.54],
       [ 0.43,  0.31,  0.17,  4.46,  4.57],
       [ 0.17,  0.03, -0.14,  4.43,  4.55],
       [ 3.15,  3.31,  3.62,  2.54,  2.43]])

In [24]:
print(f'2번 상품의 예상 선호도 : {A_new[6][1]}')
print(f'4번 상품의 예상 선호도 : {A_new[6][3]}')

2번 상품의 예상 선호도 : 3.31
4번 상품의 예상 선호도 : 2.54


***

### 3. 잠재적 요인 기반 추천시스템 구현

<br>
● 앞에서 본 예제가 사실 SVD를 이용하여 행렬의 잠재된 요인을 추출하는 방식<br>
    : User-Item 행렬 (A) -> User Latent 행렬 (US) X Item Latent 행렬 (Vt)<br>

<br>
● 그런데 결측치가 있는 행렬에 대해서는 어떻게 해야할까?<br>
1. 임의의 User Latent Matrix와 Item Latent Matrix를 만든 후<br>
2. 두 행렬 곱해서 원래의 행렬(A)과 같은 사이즈의 행렬(AA)생성<br>
3. A에서 결측치가 없는 원소들과 같은 위치에 있는 A’의 값들 사이의 차이 계산 후<br>
    계속 업데이트 (Graident Descent 적용)  


In [31]:
# 원본 행렬 A 생성
A = np.array([
    [4, np.NaN, np.NaN, 2, np.NaN],
    [np.NaN, 5, np.NaN, 3, 1],
    [np.NaN, np.NaN, 3, 4, 4],
    [5, 2, 1, 2, np.NaN]
])

num_users, num_items = A.shape

print(A)
print()

print(f'num_users:{num_users} , num_items:{num_items}')

[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]

num_users:4 , num_items:5


In [17]:
# 잠재적 요인 factor k 설정
k = 3

# User Latent Matrix (P)와 Item Latent Matrix (Q) 랜덤하게 생성
np.random.seed(1) # 난수 시드 생성

P = np.random.normal(scale=1./k, size=(num_users, k))
Q = np.random.normal(scale=1./k, size=(num_items, k)) #나중에 transpose로

print('User Latent Marix P')
print(P)
print()

print('Item Latent Matrix Q')
print(Q)

User Latent Marix P
[[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]]

Item Latent Matrix Q
[[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]


In [18]:
from sklearn.metrics import mean_squared_error

def get_rmse(A, P, Q, non_zeros) :
    # P와 Qt 곱해서 A와 비교할 행렬 AA 생성
    AA = np.dot(P, Q.T)
    
    # 실제 A 행렬에서 NaN값이 아닌 값들의 인덱스 위치와 값을 추출해서 A와 A_간의 RMSE 비교
    x_idx = [non_zero[0] for non_zero in non_zeros]
    y_idx = [non_zero[1] for non_zero in non_zeros]
    
    A_non_zeros = A[x_idx, y_idx]
    AA_non_zeros = AA[x_idx, y_idx]
    
    mse = mean_squared_error(A_non_zeros, AA_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [19]:
non_zeros = [(i, j, A[i,j]) for i in range(num_users) for j in range(num_items) if A[i,j]>0]

non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [20]:
steps = 1000

learning_rate = 0.01
r_lambda = 0.01

# Stochastic Gradient Descent with L2 regulazation
for step in range(steps) :
    for i, j, v in non_zeros :
        error = v - np.dot(P[i,:], Q[j,:].T)
        
        P[i, :] = P[i, :] + learning_rate*(2*error*Q[j,:] - r_lambda*P[i,:])
        Q[j, :] = Q[j, :] + learning_rate*(2*error*P[i,:] - r_lambda*Q[j,:])
    
    rmse = get_rmse(A,P,Q,non_zeros)
    
    if (step%50) == 0 :
        print(f'# iteration step : {step} / RMSE = {rmse}')

# iteration step : 0 / RMSE = 3.214431343870807
# iteration step : 50 / RMSE = 0.14824054147474272
# iteration step : 100 / RMSE = 0.03790852279750706
# iteration step : 150 / RMSE = 0.015828020219896898
# iteration step : 200 / RMSE = 0.010147056444467646
# iteration step : 250 / RMSE = 0.008809388563758826
# iteration step : 300 / RMSE = 0.00847815405274543
# iteration step : 350 / RMSE = 0.008381462722064694
# iteration step : 400 / RMSE = 0.008344059482524232
# iteration step : 450 / RMSE = 0.008321055210032438
# iteration step : 500 / RMSE = 0.008300334153462896
# iteration step : 550 / RMSE = 0.008278929587292172
# iteration step : 600 / RMSE = 0.008256444590137664
# iteration step : 650 / RMSE = 0.008233117192277158
# iteration step : 700 / RMSE = 0.008209266232613598
# iteration step : 750 / RMSE = 0.008185154634935837
# iteration step : 800 / RMSE = 0.008160973301317875
# iteration step : 850 / RMSE = 0.008136854578479484
# iteration step : 900 / RMSE = 0.008112888872541121
# 

In [21]:
final_AA = P@Q.T
print('예측행렬')
print(final_AA)

예측행렬
[[3.99531167 0.79083759 1.34148298 2.00094161 1.72796667]
 [6.65820263 4.98907406 0.9664588  2.9904398  1.00142118]
 [6.90782875 0.46237857 2.99352465 3.98862283 3.99335104]
 [4.98411849 2.00157185 1.00303505 2.00809952 1.15625905]]


In [22]:
print('원본행렬')
print(A)

원본행렬
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]
